In [86]:
from qiskit import *
from numpy import pi
import numpy as np
simulator = Aer.get_backend('qasm_simulator')

Для приготовление однокубитного состояния вида $|\psi\rangle=\sin \theta/2 |0\rangle+e^{i \varphi} \cos \theta/2|1\rangle$ мы будем использовать следующие операторы:

\begin{equation}
R_z (\gamma) =
\begin{pmatrix} 
e^{-i \gamma/2 }& 0\\
0 & e^{i \gamma/2 }\\
\end{pmatrix},    
\end{equation}



\begin{equation}
S =
\begin{pmatrix} 
1& 0\\
0 & i\\
\end{pmatrix}.    
\end{equation}


Рассмотрим их действие на $| 0 \rangle$ в следующем порядке:

\begin{equation}
H R_z (\delta)  | 0 \rangle = \dfrac{1}{\sqrt{2}}  \begin{pmatrix} 
1& 1\\
1 & -1\\
\end{pmatrix} \left( \!{\begin{array}{*{20}{c}}
e^{-i \delta /2} \\
0
 \end{array}} \!  \right)   = \dfrac{e^{-i \delta /2}}{\sqrt{2}}  \left( \!{\begin{array}{*{20}{c}}
1 \\
1
 \end{array}} \!  \right) 
\end{equation}

\begin{equation}
R_z (\gamma) \left[\dfrac{e^{-i \delta /2}}{\sqrt{2}}  \left( \!{\begin{array}{*{20}{c}}
1 \\
1
 \end{array}} \!  \right) \right]  =\dfrac{e^{-i \delta /2}}{\sqrt{2}}  \left( \!{\begin{array}{*{20}{c}}
e^{-i \gamma/2 } \\
e^{i \gamma/2 }
 \end{array}} \!  \right) 
\end{equation}

\begin{equation}
H \left[\dfrac{e^{-i \delta /2}}{\sqrt{2}}  \left( \!{\begin{array}{*{20}{c}}
e^{-i \gamma/2 } \\
e^{i \gamma/2 }
 \end{array}} \!  \right)  \right]  = e^{-i \delta /2} \left( \!{\begin{array}{*{20}{c}}
\cos \gamma/2  \\
-i \sin \gamma/2 
 \end{array}} \!  \right) 
\end{equation}

\begin{equation}
S \left[e^{-i \delta /2} \left( \!{\begin{array}{*{20}{c}}
\cos \gamma/2  \\
-i \sin \gamma/2 
 \end{array}} \!  \right)  \right]  = e^{-i \delta /2} \left( \!{\begin{array}{*{20}{c}}
\cos \gamma/2  \\
\sin \gamma/2 
 \end{array}} \!  \right) 
\end{equation}

\begin{equation}
R_z (\beta)\left[e^{-i \delta /2} \left( \!{\begin{array}{*{20}{c}}
\cos \gamma/2  \\
\sin \gamma/2 
 \end{array}} \!  \right)  \right]  = 
 \left( \!{\begin{array}{*{20}{c}}
e^{i (-\delta+\beta) /2} \cos \gamma/2  \\
e^{i (-\delta-\beta) /2} \sin \gamma/2 
 \end{array}} \!  \right) 
\end{equation}

Если мы выберем углы в форме $\delta=-\varphi;~\beta=\varphi$, $\gamma = - \theta - \pi$ то получим нужное состояние.


In [87]:
####################
#####PARAMETERS#####
####################
####################
phi=pi/2
theta=pi/2
gamma=-theta-pi
num_of_shots=1000

In [88]:
qreg_q = QuantumRegister(1, 'q')
creg_c = ClassicalRegister(1, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)
circuit.reset(qreg_q[0])
circuit.rz(-phi, qreg_q[0])
circuit.h(qreg_q[0])
circuit.rz(gamma,qreg_q[0])
circuit.h(qreg_q[0])
circuit.s( qreg_q[0])
circuit.rz(phi, qreg_q[0])
circuit.measure(qreg_q[0], creg_c[0])
circuit.draw()

┌───────────┐┌───┐┌────────────┐┌───┐┌───┐┌──────────┐┌─┐
q_0: ─|0>─┤ RZ(-pi/2) ├┤ H ├┤ RZ(-3pi/2) ├┤ H ├┤ S ├┤ RZ(pi/2) ├┤M├
          └───────────┘└───┘└────────────┘└───┘└───┘└──────────┘└╥┘
c: 1/════════════════════════════════════════════════════════════╩═
                                                                 0

In [89]:
simulator = Aer.get_backend('qasm_simulator')
job = execute(circuit, simulator, shots=num_of_shots)
result = job.result()
counts = result.get_counts(circuit)

Рассмотрим, как действует стандартный оператор 


$U 3(\theta, \phi, \lambda)=\left(\begin{array}{cc}\cos \left(\frac{\gamma}{2}\right) & -e^{i \lambda} \sin \left(\frac{\gamma}{2}\right) \\ e^{i \phi} \sin \left(\frac{\gamma}{2}\right) & e^{i(\phi+\lambda)} \cos \left(\frac{\gamma}{2}\right)\end{array}\right)$

При действии на нулевое состояние получим

$|\psi\rangle=\cos \gamma/2 |0\rangle+e^{i \varphi} \sin \gamma/2|1\rangle$


In [90]:
qreg_q_1 = QuantumRegister(1, 'q')
creg_c_1 = ClassicalRegister(1, 'c')
circuit_1 = QuantumCircuit(qreg_q_1, creg_c_1)

circuit_1.reset(qreg_q_1[0])
circuit_1.u3(gamma, phi, 0, qreg_q_1[0])
circuit_1.measure(qreg_q_1[0], creg_c_1[0])
circuit_1.draw()

┌───────────────────┐┌─┐
q_0: ─|0>─┤ U3(-3pi/2,pi/2,0) ├┤M├
          └───────────────────┘└╥┘
c: 1/═══════════════════════════╩═
                                0

In [91]:
job = execute(circuit_1, simulator, shots=num_of_shots)
result_1 = job.result()
counts_1 = result_1.get_counts(circuit_1)

Сравнение действия стандартного оператора и оператора, который мы приготовили самостоятельно:

In [92]:
print("\nFor self-made operator total count for 00 and 11 are:",counts)

print("\nFor standard operator u_3 total count for 00 and 11 are:",counts_1)


For self-made operator total count for 00 and 11 are: {'0': 512, '1': 488}

For standard operator u_3 total count for 00 and 11 are: {'0': 504, '1': 496}
